In [1]:
import mysql.connector
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="admin",
        password="12345678",
        database="parking"
    )

In [ ]:
from flask import Flask , request, jsonify
import os
import base64
from io import BytesIO
from PIL import Image
from ultralytics import YOLO
import cv2 
from datetime import datetime, timezone
from zoneinfo import ZoneInfo 
import json
import matplotlib.pyplot as plt

app=Flask(__name__)
def calcularPrecio(hora):

    hora = datetime.strptime(hora, "%H:%M:%S")
  
    precio  = (hora.hour * 60 + hora.minute) * 0.03
    return precio
@app.route("/borrarCliente", methods=["DELETE"])
def borrarCliente():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        usuario = data.get("usuario")
        if not usuario:
            return jsonify({"error": "usuario en blanco"}), 400
        
        cursor.execute(
            "SELECT * FROM clientes_parking WHERE usuario = %s", 
            (usuario,)
        )
        usuario_buscado = cursor.fetchone()
        if not usuario_buscado:
            return jsonify({"error": "El usuario no existe"}), 400
        
        cursor.execute(
            "DELETE FROM clientes_parking WHERE usuario = %s",
            (usuario,)
        )
        conn.commit()
        cursor.close()
        conn.close()

        return jsonify({"message": "Usuario eliminado exitosamente"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
@app.route("/actualizar_cliente", methods=["PUT"])
def actualizarCliente():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        usuario = data.get("usuario")
        matricula = data.get("matricula")
        if not usuario:
            return jsonify({"error": "usuario en blanco"}), 400
        
        cursor.execute(
            "SELECT * FROM clientes_parking WHERE usuario = %s", 
            (usuario,)
        )
        usuario_buscado = cursor.fetchone()
        if not usuario_buscado:
            return jsonify({"error": "El usuario no existe"}), 400
        
        cursor.execute(
            "UPDATE clientes_parking SET matriculas = %s WHERE usuario = %s",
            (json.dumps(matricula), usuario)
        )
        conn.commit()
        cursor.close()
        conn.close()

        return jsonify({"message": "Usuario actualizado exitosamente"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
@app.route("/borrarClienteParking", methods=["DELETE"])
def borrarClienteParking():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        id_data = data.get("id")
        if not id_data:
            return jsonify({"error": "ID en blanco"}), 400
       
        
        cursor.execute(
            "SELECT * FROM parking_tienda WHERE id = %s", 
            (id_data,)
        )
        matricula_buscada = cursor.fetchone()
        if not matricula_buscada:
            return jsonify({"error": "El registro no existe"}), 400
        
        cursor.execute(
            "DELETE FROM parking_tienda WHERE id = %s",
            (id_data,)
        )
        conn.commit()
        cursor.close()
        conn.close()

        return jsonify({"message": "Usuario eliminado exitosamente"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
@app.route("/actualizar_datos_parking_cliente", methods=["PUT"])
def actualizarDatosClienteParking():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json

        id_data= data.get("id")  
        hora_entrada = data.get("hora_entrada")
        hora_salida = data.get("hora_salida")
        precio = data.get("precio")

        if not id_data:
            return jsonify({"error": "ID no existe"}), 400

        cursor.execute("SELECT * FROM parking_tienda WHERE id = %s", (id_data,))
        cliente_obtenido= cursor.fetchone()
        if not cliente_obtenido:
            return jsonify({"error": "El registro con ese ID no existe"}), 400

        cursor.execute(
            "UPDATE parking_tienda SET hora_entrada = %s, hora_salida = %s, precio = %s WHERE id = %s",
            (hora_entrada, hora_salida, precio, id_data)
        )
        conn.commit()
        cursor.close()
        conn.close()

        return jsonify({"message": " El registro actualizado exitosamente"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route("/obtener_clientes", methods=["GET"])
def obtenerClientes():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT * FROM clientes_parking")
        clientes = cursor.fetchall()
        cursor.close()
        conn.close()

        return jsonify({"datos" : clientes}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@app.route("/obtener_datos_parking", methods=["GET"])
def obtenerDatosClientesParking():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        cursor.execute("SELECT * FROM parking_tienda")
        datos = cursor.fetchall()
        cursor.close()
        conn.close()

        return jsonify({"datos" : datos}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/agregar_usuario", methods=["POST"])
def agregarUsuario():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        matricula = data.get("matricula")
        usuario = data.get("usuario")
        if not usuario:
            return jsonify({"error": "usuario en blanco"}), 400
        
        cursor.execute(
            "SELECT * FROM clientes_parking WHERE usuario = %s", 
            (usuario,)
        )
        usuario_buscado = cursor.fetchone()
        if usuario_buscado:
            return jsonify({"error": "El usuario ya existe"}), 400
        
        cursor.execute(
            "INSERT INTO clientes_parking (usuario, matriculas) VALUES (%s, %s)",
            (usuario, json.dumps(matricula))
        )
        conn.commit()
        cursor.close()
        conn.close()

        return jsonify({"message": "Usuario agregado exitosamente"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@app.route("/parking_entrada", methods=["POST"]) 
def entradaParking():
    try:
        
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        base64_string = data.get("imagen")
        actual_path = os.path.abspath(os.getcwd())
        if not base64_string :
            return jsonify({"error": "Faltan datos (imagen)"}), 400

        # Decodificar Base64 a imagen
        image_bytes = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_bytes))
        image_path = os.path.join(actual_path, "imagen_procesada.png")
        image.save(image_path, "PNG")
        

        model_path=actual_path+'/models/get-plate-model.pt'
        model=YOLO(model_path)
        source_image=image_path
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=source_image,
            conf=0.25,
            save=False,
            verbose=False
        )   

        orig_img=cv2.imread(source_image)
        orig_img=cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        highest_conf = 0

        for box in predictions.boxes:
            conf = float(box.conf[0])
            if conf > highest_conf:
                highest_conf = conf
                plate_box = box


        x1, y1, x2, y2 = plate_box.xyxy[0]

        # Convertir a int para dibujar con OpenCV
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            #print(x1, y1, x2, y2)

        # ID de la clase detectada
        cls_id = int(plate_box.cls[0])

        # Confianza de la detección
        conf = float(plate_box.conf[0])

        # Nombre de la clase (basado en model.names)
        class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

        #print(f"Clase: {class_name}, confianza: {conf:.2f}")
        # Dibujo del rectángulo
        color = (0, 255, 0)  # Verde
        thickness = 2
        cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

        # Texto (etiqueta + confianza)
        label = f"{class_name} {conf:.2f}"
        # Para dibujar el fondo del texto (opcional, para que sea legible)
        (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        # Caja para el texto encima del bounding plate_box
        cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
        cv2.putText(orig_img, label, (x1, y1 - baseline),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        img_NZ_bgr = cv2.imread(source_image, cv2.IMREAD_COLOR)
        img_NZ_rgb = img_NZ_bgr[:, :, ::-1]   # a RGB

        plate_original = img_NZ_rgb[y1:y2,x1:x2].copy()
        cv2.imwrite(actual_path+'/testing/new_plate.png',plate_original)
        model_path = actual_path+'/models/slice-plates.pt'

        model=YOLO(model_path)
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=plate_original,
            conf=0.25,    # Umbral de confianza
            save=False,   # No que dibuje YOLO; lo haremos manualmente con OpenCV
            verbose=False
        )
        orig_img=cv2.cvtColor(plate_original, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        sorted_boxes = sorted(predictions.boxes, key=lambda box: box.xyxy[0][0])
        matricula = ""
        matricula_predecida= ""
        for plate_box in sorted_boxes:
            x1, y1, x2, y2 = plate_box.xyxy[0]

            # Convertir a int para dibujar con OpenCV
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # ID de la clase detectada
            cls_id = int(plate_box.cls[0])

            # Confianza de la detección
            conf = float(plate_box.conf[0])

            # Nombre de la clase (basado en model.names)
            class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

            matricula_predecida += class_name
            # Dibujo del rectángulo
            color = (0, 255, 0)  # Verdetual
            thickness = 2
            cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

            # Texto (etiqueta + confianza)
            label = f"{class_name} {conf:.2f}"
            # Para dibujar el fondo del texto (opcional, para que sea legible)
            (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            # Caja para el texto encima del bounding plate_box
            cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
            cv2.putText(orig_img, label, (x1, y1 - baseline),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
          
       
        matricula = matricula_predecida
        cursor.execute("SELECT usuario , id FROM clientes_parking  WHERE JSON_CONTAINS(matriculas, %s)" ,
        (json.dumps([matricula]),))
        buscar_matricula = cursor.fetchone()
        get_dateTime=  datetime.now(ZoneInfo("Europe/Madrid"))
        fecha = str(get_dateTime.date())
        hora = get_dateTime.strftime("%H:%M:%S")
        if buscar_matricula:
            cursor.execute(
            "INSERT INTO parking_tienda (matricula, hora_entrada,fecha_entrada,usuario,cliente_id) VALUES (%s, %s, %s,%s,%s)",
            (matricula, hora, fecha, buscar_matricula["usuario"],buscar_matricula["id"]))
        else:
            cursor.execute(
                    "SELECT usuario, id from clientes_parking where usuario = %s",
                    ("desconocido",))
            buscar_matricula = cursor.fetchone()

            cursor.execute(
                "INSERT INTO parking_tienda (matricula, hora_entrada,fecha_entrada,usuario,cliente_id) VALUES (%s, %s, %s,%s,%s)",
                (matricula, hora, fecha, buscar_matricula["usuario"],buscar_matricula["id"]))
            conn.commit()
            cursor.close()
            conn.close()

            return jsonify({"menssage": "matricula agrega de un usuario desconocido"}), 200
        conn.commit()
        cursor.close()
        conn.close()

        return jsonify({"message": f"matricula agregada del usuario {buscar_matricula['usuario']}"}), 200


    except Exception as e:
        return jsonify({"error": str(e)}), 500
    

@app.route("/parking_salida", methods=["POST"])
def salidaParking():
    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        base64_string = data.get("imagen")
        actual_path = os.path.abspath(os.getcwd())
        if not base64_string :
            return jsonify({"error": "Faltan datos (imagen)"}), 400

        # Decodificar Base64 a imagen
        image_bytes = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_bytes))
        image_path = os.path.join(actual_path, "imagen_procesada.png")
        image.save(image_path, "PNG")
        

        model_path=actual_path+'/models/get-plate-model.pt'
        model=YOLO(model_path)
        source_image=image_path
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=source_image,
            conf=0.25,
            save=False,
            verbose=False
        )   

        orig_img=cv2.imread(source_image)
        orig_img=cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        highest_conf = 0

        for box in predictions.boxes:
            conf = float(box.conf[0])
            if conf > highest_conf:
                highest_conf = conf
                plate_box = box


        x1, y1, x2, y2 = plate_box.xyxy[0]

        # Convertir a int para dibujar con OpenCV
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            #print(x1, y1, x2, y2)

        # ID de la clase detectada
        cls_id = int(plate_box.cls[0])

        # Confianza de la detección
        conf = float(plate_box.conf[0])

        # Nombre de la clase (basado en model.names)
        class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

        #print(f"Clase: {class_name}, confianza: {conf:.2f}")
        # Dibujo del rectángulo
        color = (0, 255, 0)  # Verde
        thickness = 2
        cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

        # Texto (etiqueta + confianza)
        label = f"{class_name} {conf:.2f}"
        # Para dibujar el fondo del texto (opcional, para que sea legible)
        (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        # Caja para el texto encima del bounding plate_box
        cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
        cv2.putText(orig_img, label, (x1, y1 - baseline),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        img_NZ_bgr = cv2.imread(source_image, cv2.IMREAD_COLOR)
        img_NZ_rgb = img_NZ_bgr[:, :, ::-1]   # a RGB

        plate_original = img_NZ_rgb[y1:y2,x1:x2].copy()
        cv2.imwrite(actual_path+'/testing/new_plate.png',plate_original)
        model_path = actual_path+'/models/slice-plates.pt'

        model=YOLO(model_path)
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=plate_original,
            conf=0.25,    # Umbral de confianza
            save=False,   # No que dibuje YOLO; lo haremos manualmente con OpenCV
            verbose=False
        )
        orig_img=cv2.cvtColor(plate_original, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        sorted_boxes = sorted(predictions.boxes, key=lambda box: box.xyxy[0][0])
     
        matricula_predecida= ""
        detecto_placa = False
        for plate_box in sorted_boxes:
            detecto_placa = True
            x1, y1, x2, y2 = plate_box.xyxy[0]

            # Convertir a int para dibujar con OpenCV
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # ID de la clase detectada
            cls_id = int(plate_box.cls[0])

            # Confianza de la detección
            conf = float(plate_box.conf[0])

            # Nombre de la clase (basado en model.names)
            class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

            matricula_predecida += class_name
            # Dibujo del rectángulo
            color = (0, 255, 0)  # Verdetual
            thickness = 2
            cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

            # Texto (etiqueta + confianza)
            label = f"{class_name} {conf:.2f}"
            # Para dibujar el fondo del texto (opcional, para que sea legible)
            (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            # Caja para el texto encima del bounding plate_box
            cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
            cv2.putText(orig_img, label, (x1, y1 - baseline),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        matricula = matricula_predecida
        if detecto_placa:
            cursor.execute(
            "SELECT * from parking_tienda where	matricula = %s and hora_salida is null",
            (matricula,))
            buscar_matricula = cursor.fetchall()
            if buscar_matricula:
                get_dateTime=  datetime.now(ZoneInfo("Europe/Madrid"))
                hora = get_dateTime.strftime("%H:%M:%S")
                hora_entrada = buscar_matricula[0]["hora_entrada"]
                hora_formateada =  datetime.strptime(hora_entrada, "%H:%M:%S")
                hora_formateada2 = datetime.strptime(hora, "%H:%M:%S")
                diferencia = hora_formateada2 - hora_formateada
                precio_calculado = calcularPrecio(str(diferencia))
                cursor.execute(
                "UPDATE parking_tienda  SET hora_salida = %s , precio = %s WHERE matricula = %s and hora_salida is null",
                (hora, precio_calculado ,matricula))
                conn.commit()
                cursor.close()
                conn.close()
                return jsonify({    
                    "message": "Imagen procesada y guardada en la base de datos",
                }), 200
            else:
                conn.commit()
                cursor.close()
                conn.close()
                return jsonify({"error": "No se puede salir, no se guardo correctamente o la matricula dada no existe"}), 400
                
        return jsonify({"error": "No se encontró la matrícula con hora de salida sin asignar",}), 404
      
                
    except Exception as e:
        return jsonify({"error": str(e)}), 500    

if __name__=="__main__":
    app.run()   # localhost at 5000

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Apr/2025 00:33:47] "GET /obtener_clientes HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:33:47] "GET /obtener_datos_parking HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:33:54] "DELETE /borrarClienteParking HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:33:57] "DELETE /borrarClienteParking HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:36:43] "POST /parking_entrada HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:39:07] "POST /parking_entrada HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:41:25] "POST /parking_entrada HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:42:48] "POST /parking_entrada HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 00:45:05] "POST /parking_salida HTTP/1.1" 200 -


In [ ]:
#antiguo codigo para agregar usuario y matricula

@app.route("/parking_entrada", methods=["POST"])
def entradaParking():
    try:

        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        data = request.json
        base64_string = data.get("imagen")
        if not base64_string :
            return jsonify({"error": "Faltan datos (imagen)"}), 400

        # Decodificar Base64 a imagen
        image_bytes = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_bytes))

        # Guardar la imagen como PNG
        actual_path = os.path.abspath(os.getcwd())
        image_path = os.path.join(actual_path, "imagen_procesada.png")
        image.save(image_path, "PNG")
    
        # Procesar la imagen con YOLO
        model_Path=actual_path+'/models/get-plate-model.pt'
        model=YOLO(model_Path)
        source_image=actual_path+'/testing/old-car.jpeg'
        print(source_image)
        """
        images=[
            'data/val/images/ba2b2dbd-captura_20250318-161749.png',
            'data/val/images/be627c47-captura_20250318-161845.png'
        ]
        """
        results = model.predict(
            source=source_image,
            conf=0.25,    # Umbral de confianza
            save=False,   # No que dibuje YOLO; lo haremos manualmente con OpenCV
            verbose=False
        )
        orig_img=cv2.imread(source_image)
        orig_img=cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

        predictions = results[0]
        detecto_placa = False  # Variable para verificar si hay detecciones
        for box in predictions.boxes:
            detecto_placa = True
            x1, y1, x2, y2 = box.xyxy[0]

            # Convertir a int para dibujar con OpenCV
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            print(x1, y1, x2, y2)

            # ID de la clase detectada
            cls_id = int(box.cls[0])

            # Confianza de la detección
            conf = float(box.conf[0])

            # Nombre de la clase (basado en model.names)
            class_name = model.names[cls_id] if model.names and cls_id < len(model.names) else f"cls_{cls_id}"

            print(f"Clase: {class_name}, confianza: {conf:.2f}")
            # Dibujo del rectángulo
            color = (0, 255, 0)  # Verde
            thickness = 2
            cv2.rectangle(orig_img, (x1, y1), (x2, y2), color, thickness)

            # Texto (etiqueta + confianza)
            label = f"{class_name} {conf:.2f}"
            # Para dibujar el fondo del texto (opcional, para que sea legible)
            (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            # Caja para el texto encima del bounding box
            cv2.rectangle(orig_img, (x1, y1 - th - baseline), (x1 + tw, y1), color, -1)
            cv2.putText(orig_img, label, (x1, y1 - baseline),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

            # Si se detecta una caja, se recorta la imagen
            img_NZ_bgr = cv2.imread(source_image, cv2.IMREAD_COLOR)
            img_NZ_rgb = img_NZ_bgr[:, :, ::-1]  # a RGB

            plate = img_NZ_rgb[y1:y2, x1:x2].copy()
            cv2.imwrite(actual_path + '/testing/new_plate224.png', plate)
            matricula = "9876SND"  
            cursor.execute("SELECT usuario , id FROM clientes_parking  WHERE JSON_CONTAINS(matriculas, %s)" ,
            (json.dumps([matricula]),))
            buscar_matricula = cursor.fetchone()
            get_dateTime=  datetime.now(ZoneInfo("Europe/Madrid"))
            fecha = str(get_dateTime.date())
            hora = get_dateTime.strftime("%H:%M:%S")
            if buscar_matricula:
        
                if detecto_placa:
                    cursor.execute(
                    "INSERT INTO parking_tienda (matricula, hora_entrada,fecha_entrada,usuario,cliente_id) VALUES (%s, %s, %s,%s,%s)",
                    (matricula, hora, fecha, buscar_matricula["usuario"],buscar_matricula["id"])
                )
                
            else:
                cursor.execute(
                    "SELECT usuario, id from clientes_parking where usuario = %s",
                    ("desconocido",))
                buscar_matricula = cursor.fetchone()

                cursor.execute(
                    "INSERT INTO parking_tienda (matricula, hora_entrada,fecha_entrada,usuario,cliente_id) VALUES (%s, %s, %s,%s,%s)",
                    (matricula, hora, fecha, buscar_matricula["usuario"],buscar_matricula["id"]))
                conn.commit()
                cursor.close()
                conn.close()

                return jsonify({"menssage": "matricula agrega de un usuario desconocido"}), 200
            conn.commit()

            cursor.close()
            conn.close()

        return jsonify({    
            "message": "Imagen procesada y guardada en la base de datos",
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500